# Load Data - Azure Sql Database - From Azure Blob Storage

**Step 1 - Create a table and schema**


In [ ]:
IF SCHEMA_ID('BlobTest') IS NULL
EXEC('CREATE SCHEMA [BlobTest]')

IF NOT EXISTS ( SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[BlobTest].[FileFromBlobDistination]') AND type IN ( N'U' ))
    BEGIN
    CREATE TABLE BlobTest.FileFromBlobDistination ( CONTENT NVARCHAR(MAX) NULL );
    END;


**Step 2 - Create Master Key**

In [ ]:
IF NOT EXISTS ( SELECT * FROM sys.symmetric_keys WHERE name LIKE '%DatabaseMasterKey%' )
    BEGIN
    PRINT 'CREATE MASTER KEY ENCRYPTION BY PASSWORD = NicePassword;';
    END;

**Step 3 - Create a Database scoped credential**
> Note: the ? at the beginning of the SAS token should be removed

In [ ]:
IF NOT EXISTS ( SELECT *
FROM sys.database_scoped_credentials
WHERE name IN ( N'https://3marketanalysis.blob.core.windows.net/tradesupport/' ))
    BEGIN
    CREATE DATABASE SCOPED
    CREDENTIAL [https://3marketanalysis.blob.core.windows.net/tradesupport/]
    WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
    SECRET = 'sv=2019-10-10&st=2020-11-02T08%3A10%3A53Z&se=2020-11-30T08%3A10%3A00Z&sr=c&sp=racwdl&sig=D2afwKDc2LT%2Bb5ZNIkDYDTYoeyEpORhucFEBjP4xfn4%3D'
END;

**Step 4 - Create an external data source to the container**

> Note: Location doesn't have a trailing /, even through the CREDENTIALS does.

In [ ]:
IF NOT EXISTS (SELECT *
FROM sys.external_data_sources
WHERE name = 'tradesupport')
BEGIN
    CREATE EXTERNAL DATA SOURCE tradesupport 
WITH
(
    TYPE=BLOB_STORAGE, 
    LOCATION='https://3marketanalysis.blob.core.windows.net/tradesupport', 
    CREDENTIAL=[https://3marketanalysis.blob.core.windows.net/tradesupport/]
);
END

**Step 5 - Bulk insert data from blob storage**

**Final Step - Clean up**

In [ ]:
IF EXISTS ( SELECT *
FROM sys.objects
WHERE object_id = OBJECT_ID(N'[BlobTest].[FileFromBlobDistination]') AND type IN ( N'U' ))
	BEGIN
    DROP TABLE [BlobTest].[FileFromBlobDistination]
END

DROP SCHEMA IF EXISTS [BlobTest];

In [ ]:
IF EXISTS ( SELECT *
FROM sys.database_scoped_credentials
WHERE name IN ( N'https://3marketanalysis.blob.core.windows.net/tradesupport/' ))
    BEGIN
    DROP DATABASE SCOPED
    CREDENTIAL [https://3marketanalysis.blob.core.windows.net/tradesupport/]
END;

In [ ]:

IF EXISTS (SELECT * FROM sys.external_data_sources WHERE name = 'tradesupport')
BEGIN 
DROP EXTERNAL DATA SOURCE tradesupport
END 